In [16]:
import pandas as pd

In [18]:
df = pd.read_csv("../../results/meta-llama/Meta-Llama-3.1-8B-Instruct/error/small_explanation/2024-08-12-14-59-11_explanation/checkpoint-147/first_iteration/2024-08-12-19-33-21_explanation/checkpoint-238/second_iteration/checkpoint-154/third_iteration/validation_results.csv")
#sort by f1
df = df.sort_values(by='f1', ascending=False)
df


,Unnamed: 0,checkpoint_path,checkpoint_number,epoch,f1,precision,recall
0,0,../../results/meta-llama/Meta-Llama-3.1-8B-Ins...,79,1,0.728571,0.743750,0.714
1,1,../../results/meta-llama/Meta-Llama-3.1-8B-Ins...,158,2,0.711009,0.833333,0.620
5,5,../../results/meta-llama/Meta-Llama-3.1-8B-Ins...,475,6,0.703196,0.819149,0.616
6,6,../../results/meta-llama/Meta-Llama-3.1-8B-Ins...,554,7,0.700565,0.805195,0.620
7,7,../../results/meta-llama/Meta-Llama-3.1-8B-Ins...,634,8,0.693491,0.849275,0.586
9,9,../../results/meta-llama/Meta-Llama-3.1-8B-Ins...,790,10,0.690531,0.816940,0.598
8,8,../../results/meta-llama/Meta-Llama-3.1-8B-Ins...,713,9,0.682984,0.818436,0.586
4,4,../../results/meta-llama/Meta-Llama-3.1-8B-Ins...,396,5,0.655215,0.847619,0.534
2,2,../../results/meta-llama/Meta-Llama-3.1-8B-Ins...,237,3,0.630907,0.872792,0.494
3,3,../../results/meta-llama/Meta-Llama-3.1-8B-Ins...,317,4,0.621656,0.856140,0.488


In [19]:
def create_prompt_with_explanation(product1: str, product2: str, answer: int):
    prompt = f"Do the two entity descriptions refer to the same real-world entity? Entity 1: '{product1}'. Entity 2: '{product2}'."

    return  {
        "prompt": prompt,
        "completion": answer
    }

In [20]:
validated_df = pd.read_json("../../results/meta-llama/Meta-Llama-3.1-8B-Instruct/error/small_explanation/2024-08-12-14-59-11_explanation/checkpoint-147/first_iteration/2024-08-12-19-33-21_explanation/checkpoint-238/second_iteration/checkpoint-154/third_iteration/checkpoint-79/validation_results.json")
validation_with_explanations_df = pd.read_csv("../../data/wdc/preprocessed_wdcproducts80cc20rnd000un_valid_small_40_mini.csv")
training_simple_df = pd.read_csv("../../data/wdc/preprocessed_wdcproducts80cc20rnd000un_train_small_explanations_40_mini_simple.csv")

In [21]:
# drop all columns from validation_with_explanations_df except pair_id and content
validation_with_explanations_df = validation_with_explanations_df[["pair_id", "content"]]

# merge the two based on pair id 
merged_df = validated_df.merge(validation_with_explanations_df, on="pair_id")

In [22]:
def extract_most_important_attribute(data_string):
    # Split the string into individual attributes
    attributes = data_string.split('\n')

    # Initialize variables to track the most important attribute
    max_importance = -1
    most_important_attribute = ""

    # Loop through each attribute
    for attribute in attributes:
        if 'importance=' in attribute:
            # Extract the importance value
            try:
                importance_str = attribute.split('importance=')[1].split('|||')[0]
                importance = float(importance_str)
            except:
                print(f"Error converting importance value to float. importance_str: {attribute}")
                importance = -1

            # Check if this is the most important attribute so far
            if importance > max_importance:
                max_importance = importance
                try:
                    most_important_attribute = attribute.split('attribute=')[1].split('|||')[0]
                except:
                    print(f"Error extracting most important attribute. attribute: {attribute}")
                    most_important_attribute = ""

    return most_important_attribute

In [23]:
# apply the extract_most_important_attribute function to the content column
merged_df["most_important_attribute"] = merged_df["content"].apply(extract_most_important_attribute)

Error converting importance value to float. importance_str: attribute=game title|||importance=missing|||values=missing###Pokemon S&S2: Rebel Clash|||similarity=0.00  
Error converting importance value to float. importance_str: attribute=type|||importance=missing|||values=missing###Trading Card Game|||similarity=0.00  


In [24]:
miss_classified_df = merged_df[merged_df["chatbot_response_clean"] != merged_df["label"]]
miss_classified_df["most_important_attribute"].value_counts()

most_important_attribute
model                         157
brand                          23
type                           18
storage capacity               10
product type                    7
memory type                     5
size                            4
socket type                     3
speed                           3
prints                          2
channel count                   2
connection type                 2
ball size                       2
memory capacity                 2
series                          2
pack size                       1
usage                           1
macro capability                1
number of shots                 1
form factor                     1
expansion set                   1
string gauge                    1
case type                       1
sheets count                    1
wireless standard               1
number of exposures/photos      1
water resistance                1
resolution                      1
managed                

In [25]:
training_large_df = pd.read_csv("../../data/wdc/preprocessed_wdcproducts80cc20rnd000un_train_large_40_mini.csv")
# apply the extract_most_important_attribute function to the content column
training_large_df["most_important_attribute"] = training_large_df["content"].apply(extract_most_important_attribute)
training_large_df["most_important_attribute"].value_counts()

Error extracting most important attribute. attribute: attribute/model|||importance=0.95|||values=HyperX Fury 4GB DDR3 1600MHz CL10 Ram###4GB DDR3 1600MHz HyperX Fury Black Series|||similarity=0.90  
Error extracting most important attribute. attribute: attribute/socket type|||importance=0.9|||values=AM4###AM4|||similarity=1.00  
Error extracting most important attribute. attribute: attribute/socket type|||importance=0.10|||values=AM4###AM4|||similarity=1.00  
Error converting importance value to float. importance_str: attribute=frequency|||importance=missing|||values=missing###2666MHz|||similarity=missing  
Error converting importance value to float. importance_str: attribute=memory type|||importance=missing|||values=missing###DDR4|||similarity=missing  
Error extracting most important attribute. attribute: attribute/storage capacity|||importance=0.9|||values=32GB###32GB|||similarity=1.00  
Error extracting most important attribute. attribute: attribute/storage capacity|||importance=0.

most_important_attribute
model                  12114
brand                   2744
type                     993
storage capacity         731
product type             437
                       ...  
supported processor        1
cooling type               1
infrared burner            1
UHS class                  1
standards                  1
Name: count, Length: 295, dtype: int64

In [26]:
print(miss_classified_df.iloc[0][["title_left", "title_right", "label", "content"]])

title_left                          HyperX Fury DDR3 1600MHz 8GB
title_right    Memoria DDR3 1600Mhz Kit 8GB HyperX Kingston (...
label                                                          0
content        No.  \nattribute=brand|||importance=0.05|||val...
Name: 2, dtype: object


In [27]:
title_left = miss_classified_df.iloc[0]["title_left"]
title_right = miss_classified_df.iloc[0]["title_right"]
label = miss_classified_df.iloc[0]["label"]
content = miss_classified_df.iloc[0]["content"]

example = {
    "title_left": title_left,
    "title_right": title_right,
    "label": label,
    "content": content
}

print(example)

{'title_left': 'HyperX Fury DDR3 1600MHz 8GB', 'title_right': 'Memoria DDR3 1600Mhz Kit 8GB HyperX Kingston (2x4GB)', 'label': 0, 'content': 'No.  \nattribute=brand|||importance=0.05|||values=HyperX###HyperX|||similarity=1.00  \nattribute=model|||importance=-0.90|||values=Fury|||Fury Kit|||similarity=0.50  \nattribute=memory type|||importance=0.10|||values=DDR3###DDR3|||similarity=1.00  \nattribute=memory speed|||importance=0.10|||values=1600MHz###1600MHz|||similarity=1.00  \nattribute=memory size|||importance=-0.80|||values=8GB###8GB (2x4GB)|||similarity=0.75  \nattribute=configuration|||importance=-0.95|||values=Single Module###Dual Channel (2x4GB)|||similarity=0.00  '}


In [28]:
# for every entry in the miss_classified_df take 5 examples from the training_large_df that have the same most_important_attribute and the same label
# Assuming your dataframes are named miss_classified_df and training_large_df
selected_examples = []

# Iterate over each row in miss_classified_df
for idx, row in miss_classified_df.iterrows():
    # Get the most important attribute and label for the current row
    attribute = row['most_important_attribute']
    label = row['label']
    
    # Find 5 matching examples in training_large_df
    matching_examples = training_large_df[
        (training_large_df['most_important_attribute'] == attribute) & 
        (training_large_df['label'] == label)
    ].head(5)
    
    # Append the matching examples to the list
    selected_examples.append(matching_examples)

# Combine all selected examples into a single DataFrame
selected_examples_df = pd.concat(selected_examples, ignore_index=True)


In [29]:
# drop duplicat pai_ids 
selected_examples_df = selected_examples_df.drop_duplicates(subset=['pair_id'])

# convert the selected 
df_train = selected_examples_df.apply(lambda row: create_prompt_with_explanation(row["title_left"], row["title_right"], row["content"]), axis=1, result_type='expand')

df_train.columns = ['prompt', 'completion']

# concat df_train with the training_simple_df
df_train = pd.concat([df_train, training_simple_df])
df_train.to_csv("../../results/meta-llama/Meta-Llama-3.1-8B-Instruct/error/small_explanation/2024-08-12-14-59-11_explanation/checkpoint-147/first_iteration/2024-08-12-19-33-21_explanation/checkpoint-238/second_iteration/checkpoint-154/third_iteration/checkpoint-79/enhanced_training.csv", index=False)